In [1]:
import sys
sys.path.append("..")
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from torch import nn
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle

In [2]:
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("klue/bert-base")
if torch.cuda.is_available():
        p_encoder.cuda()
        q_encoder.cuda()

In [35]:
def get_relevant_doc(q_encoder, p_encoder, query, context, elastic_score_dict,k=100):
    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

        p_embs = []
        # for p in context:
        #     p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        #     p_emb = p_encoder(**p).to('cpu').numpy()
        #     p_embs.append(p_emb)
        p_seqs = tokenizer(context, padding="max_length", truncation=True, return_tensors='pt')
        dataset = TensorDataset(
            p_seqs["input_ids"],
            p_seqs["attention_mask"],
            p_seqs["token_type_ids"]
        )
        sampler = SequentialSampler(dataset)
        dataloader = DataLoader(
            dataset,
            sampler=sampler,
            batch_size=100
        )
        for _, batch in enumerate(dataloader):
            batch = tuple(t.cuda() for t in batch)

            p_inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2]
            }
            
            outputs = p_encoder(**p_inputs).to("cpu").numpy()
            p_embs.extend(outputs)

    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    #print(p_embs.size())
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    #print(dot_prod_scores[0][0])
    elastic_score = torch.Tensor([elastic_score_dict[query]])
    #weight_score = weight * dot_prod_scores
    #print(weight_score[0][0])
    temp_score = elastic_score + dot_prod_scores
    # print(dot_prod_scores)
    # print(dot_prod_scores.size())
    # print(temp_score)
    
    #rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    rank = torch.argsort(temp_score, dim=1, descending=True).squeeze()
    #print(rank)

    return_context = []
    for i in range(k):
        return_context.append(context[rank[i]])
    
    return dot_prod_scores.squeeze(), return_context #,rank[:k]

In [10]:
with open('/opt/ml/data/elastic_new_valid_500.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/elastic_new_valid_score_500.bin','rb') as f:
    elastic_valid_score = pickle.load(f)

In [11]:
querys = list(elastic_valid.keys())

In [12]:
hybrid = {}
# range(len(querys))
for i in tqdm(range(len(querys))):
    _, re_rank = get_relevant_doc(q_encoder,p_encoder,querys[i],elastic_valid[querys[i]],elastic_valid_score)
    hybrid[querys[i]] = re_rank

100%|██████████| 240/240 [08:32<00:00,  2.14s/it]


In [30]:
with open("/opt/ml/data/hybrid_valid_retrieval.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [33]:
with open('/opt/ml/data/elastic_new_train_500.bin','rb') as f:
    elastic_train = pickle.load(f)
with open('/opt/ml/data/elastic_new_train_score_500.bin','rb') as f:
    elastic_train_score = pickle.load(f)

In [36]:
querys = list(elastic_train.keys())
hybrid = {}
# range(len(querys))
for i in tqdm(range(len(querys))):
    _, re_rank = get_relevant_doc(q_encoder,p_encoder,querys[i],elastic_train[querys[i]],elastic_train_score)
    hybrid[querys[i]] = re_rank

100%|██████████| 3952/3952 [2:20:41<00:00,  2.14s/it]


In [37]:
with open("/opt/ml/data/hybrid_train_retrieval.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [50]:
with open('/opt/ml/data/elastic_retrieval_500.bin','rb') as f:
    elastic_test = pickle.load(f)
with open('/opt/ml/data/elastic_retrieval_score_500.bin','rb') as f:
    elastic_test_score = pickle.load(f)

In [51]:
querys = list(elastic_test.keys())
hybrid = {}
# range(len(querys))
for i in tqdm(range(len(querys))):
    _, re_rank = get_relevant_doc(q_encoder,p_encoder,querys[i],elastic_test[querys[i]],elastic_test_score)
    hybrid[querys[i]] = re_rank

100%|██████████| 600/600 [21:26<00:00,  2.14s/it]


In [53]:
with open("/opt/ml/data/hybrid_test_retrieval.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [76]:
num = 250
print(querys[num])
hybrid[querys[num]][0]

부피와 무게감을 나타낸 얼굴이나 짧은 목 등을 통해 알 수 있는 불상의 축조 시기는?


'경상북도 예천군 예천읍 동본동 3층석탑과 함께 전해오는 통일신라 후기의 불상으로 하나의 돌에 새겨진 전체 높이 3.46m의 거대한 석조불상이다. 머리에는 작은 소라 모양의 머리칼을 붙여 놓았으며, 정수리 부근에는 상투 모양의 머리(육계)가 큼직하게 표현되었다. 원만한 얼굴에는 길다란 눈, 짧은 코, 적당한 입이 적절하게 표현되어 부드러운 곡선의 얼굴과 함께 자비롭고 온화한 미소를 실감나게 나타내고 있다. 큰 얼굴에 비하여 작아진 상체는 굵고 짧은 목과 좁은 어깨, 짧은 팔 등이 평판적인 가슴과 함께 움츠린 듯하여 다소 위축된 느낌을 준다. 오른팔은 옆으로 내려 몸에 붙인 채 옷자락을 살짝 잡고 있으며, 왼손은 앞으로 들어 새끼 손가락을 제외한 손가락을 안으로 굽히고 있다. 양 어깨를 감싸고 입은 옷은 허벅지에서 Y자형으로 갈라지고 양 다리에서는 타원형의 주름을 만들면서 흐른다. 둔중하고 도식화된 이러한 형태의 옷주름 표현은 8세기 이후의 불상에서 흔히 볼 수 있는 모습으로 불상이 만들어진 연대를 짐작할 수 있게 해준다. 양 다리에서 있는 긴 타원형의 옷주름, 부피감 없는 둔중한 신체, 그러면서도 아직 경직화되지는 않은 얼굴 모습 등을 고려할 때 통일신라 불상양식을 계승하면서 고려시대로 넘어가는 과도기적인 작품으로 보인다.'